# Land ice Investigations

Let's start by importing modules

In [1]:
from modules import *

Now we want to check we have the right data. FOr this we have land ice data, ratiation data (LW and SW) and ice velocity data.

In [2]:
# Download data needed

Now we want to compute the correlations and regressions